# Performance Numpy vs. Cupy vs. ActionRulesDiscovery

## Parameters

In [9]:
stable_attributes = ["gender", "SeniorCitizen", "Partner"]
flexible_attributes = ["PhoneService", 
                       "InternetService", 
                       "OnlineSecurity", 
                       "DeviceProtection", 
                       "TechSupport",
                       "StreamingTV"]
target = 'Churn'
min_stable_attributes = 2
min_flexible_attributes = 1 #min 1
min_undesired_support = 50
min_undesired_confidence = 0.6
min_desired_support = 50
min_desired_confidence = 0.6
undesired_state = 'Yes'
desired_state = 'No'

## Pandas (action-rules)

In [10]:
import pandas as pd

### Data

Import data to Pandas DataFrame.

In [11]:
pd.set_option('display.max_columns', None)
dataFrame = pd.read_csv("data/telco.csv", sep=";")
dataFrame.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Action rules

In [12]:
from action_rules import ActionRules
# Action Rules Mining
action_rules = ActionRules(
    min_stable_attributes = min_stable_attributes, 
    min_flexible_attributes = min_flexible_attributes, 
    min_undesired_support = min_undesired_support, 
    min_undesired_confidence = min_undesired_confidence, 
    min_desired_support = min_desired_support,
    min_desired_confidence = min_desired_confidence, 
    verbose = False)

In [13]:
def measure_time():
    action_rules.fit(
        data = dataFrame, 
        stable_attributes = stable_attributes, 
        flexible_attributes = flexible_attributes, 
        target = target, 
        undesired_state = undesired_state,
        desired_state = desired_state, 
        use_gpu = False
    )

### Count action rules

In [14]:
measure_time()
len(action_rules.get_rules().get_ar_notation())

327

### Time

In [15]:
%timeit measure_time()

1.35 s ± 46.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Show first five rules

In [8]:
for ar in action_rules.get_rules().get_ar_notation()[:5]:
    print(ar)
    print()

{'undesired': {'itemset': (0, 3, 11, 14), 'support': 144, 'confidence': 0.602510460251046, 'target': 24}, 'desired': {'itemset': (0, 3, 11, 16), 'support': 102, 'confidence': 0.6415094339622641, 'target': 23}, 'uplift': 58.32075471698113}

{'undesired': {'itemset': (0, 3, 11, 14), 'support': 144, 'confidence': 0.602510460251046, 'target': 24}, 'desired': {'itemset': (0, 3, 13, 16), 'support': 66, 'confidence': 0.8571428571428571, 'target': 23}, 'uplift': 109.85714285714283}

{'undesired': {'itemset': (0, 3, 7, 11, 14), 'support': 130, 'confidence': 0.6074766355140186, 'target': 24}, 'desired': {'itemset': (0, 3, 7, 11, 16), 'support': 94, 'confidence': 0.6438356164383562, 'target': 23}, 'uplift': 53.780821917808225}

{'undesired': {'itemset': (0, 3, 7, 11, 14), 'support': 130, 'confidence': 0.6074766355140186, 'target': 24}, 'desired': {'itemset': (0, 3, 7, 13, 16), 'support': 62, 'confidence': 0.8493150684931506, 'target': 23}, 'uplift': 97.75342465753423}

{'undesired': {'itemset': (

## cuDF (action-rules)

In [9]:
import cudf

### Data

Import data to cuDF dataframe

In [11]:
dataFrame = cudf.read_csv("data/telco.csv", sep=";")
dataFrame.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [12]:
### Actio rules

In [13]:
from action_rules import ActionRules
# Action Rules Mining
action_rules = ActionRules(
    min_stable_attributes = min_stable_attributes, 
    min_flexible_attributes = min_flexible_attributes, 
    min_undesired_support = min_undesired_support, 
    min_undesired_confidence = min_undesired_confidence, 
    min_desired_support = min_desired_support,
    min_desired_confidence = min_desired_confidence, 
    verbose = False)

In [14]:
def measure_time_2():
    action_rules.fit(
        data = dataFrame, 
        stable_attributes = stable_attributes, 
        flexible_attributes = flexible_attributes, 
        target = target, 
        undesired_state = undesired_state,
        desired_state = desired_state, 
        use_gpu = True # Use cuDF
    )

### Count action rules

In [15]:
measure_time_2()
len(action_rules.get_rules().get_ar_notation())

227

### Time

In [16]:
%timeit measure_time_2()

5.35 s ± 76.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Show first five rules

In [17]:
for ar in action_rules.get_rules().get_ar_notation()[:5]:
    print(ar)
    print()

[(gender: Female) ∧ (SeniorCitizen: 1) ∧ (OnlineSecurity*: No) ∧ (DeviceProtection: No → Yes)] ⇒ [Churn: Churn_<item_target>_Yes → Churn_<item_target>_No], support of undesired part: 144, confidence of undesired part: 0.602510460251046, support of desired part: 102, confidence of desired part: 0.6415094339622641, uplift: 58.32075471698113

[(gender: Female) ∧ (SeniorCitizen: 1) ∧ (OnlineSecurity: No → Yes) ∧ (DeviceProtection: No → Yes)] ⇒ [Churn: Churn_<item_target>_Yes → Churn_<item_target>_No], support of undesired part: 144, confidence of undesired part: 0.602510460251046, support of desired part: 66, confidence of desired part: 0.8571428571428571, uplift: 109.85714285714283

[(gender: Female) ∧ (SeniorCitizen: 1) ∧ (PhoneService*: Yes) ∧ (OnlineSecurity*: No) ∧ (DeviceProtection: No → Yes)] ⇒ [Churn: Churn_<item_target>_Yes → Churn_<item_target>_No], support of undesired part: 130, confidence of undesired part: 0.6074766355140186, support of desired part: 94, confidence of desired

## DEAR algorithm (actionRulesDiscovery)

In [23]:
from actionrules.actionRulesDiscovery import ActionRulesDiscovery

In [24]:
### Action Rules

In [25]:
def measure_time_3():
    actionRulesDiscovery = ActionRulesDiscovery()
    actionRulesDiscovery.load_pandas(dataFrame)
    actionRulesDiscovery.fit(stable_attributes = stable_attributes ,
                             flexible_attributes = flexible_attributes,
                             consequent = target,
                             conf = min_undesired_confidence * 100, # There is just one confidence for both desired and undesired part.
                             supp = -min_undesired_support, # There is just one support for both desired and undesired part. Minus means that it is the absolute support.
                             desired_changes = [[undesired_state, desired_state]],
                             is_nan=False,
                             is_reduction=True,
                             min_stable_attributes=min_stable_attributes,
                             min_flexible_attributes=min_flexible_attributes)

### Count action rules

In [26]:
measure_time_3()
len(actionRulesDiscovery.get_action_rules())

/mnt/c/Users/lukas/OneDrive/Documents/VSE/action-rules/.venv/lib/python3.10/site-packages/cudf/core/dataframe.py:4625: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  warnings.warn(


ValueError: user defined function compilation failed.

### Time

In [ ]:
%timeit measure_time_3()

### Show first five rules

In [ ]:
for rule in actionRulesDiscovery.get_action_rules_representation()[:20]:
    print(rule)
    print(" ")